In [8]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm, datasets
from sklearn.kernel_approximation import RBFSampler, PolynomialCountSketch
from sklearn.metrics import accuracy_score as acc

In [1]:
def get_train_test_split(X, y, idx):
    return X[idx,:], y[idx], X[-idx, :], y[-idx]

In [9]:
digits = datasets.load_digits()

X = digits.data
y = digits.target

C = 1.0  # SVM regularization parameter

# the standard models are defined below
models = (svm.LinearSVC(C=C, max_iter=10000, dual = True),
          svm.SVC(kernel='rbf', gamma=0.7, C=C),
          svm.SVC(kernel='poly', degree=3, gamma=1.0, C=C)
          )

nr_comp = 1000
rbf_feature = RBFSampler(gamma=0.7, random_state=1, n_components=nr_comp)
poly_ts = PolynomialCountSketch(degree=3, gamma=1.0, random_state=1, n_components=nr_comp)

nr = 10
accs = [0 for _ in range(len(models)+2)]
for _ in range(nr):
    
    train_index = np.random.choice(X.shape[0], int(0.8*X.shape[0]), replace=False)

    X_train, y_train, X_test, y_test = get_train_test_split(X, y, train_index)
    X_train_rbf, X_test_rbf = rbf_feature.fit_transform(X_train), rbf_feature.fit_transform(X_test)
    X_train_poly, X_test_poly = poly_ts.fit_transform(X_train), poly_ts.fit_transform(X_test)

    models = [clf.fit(X_train, y_train) for clf in models] 
    
    # Linear SVM models using explicit feature maps
    model_rbf = svm.LinearSVC(C=C, max_iter=10000, dual = False).fit(X_train_rbf, y_train)
    model_ts = svm.LinearSVC(C=C, max_iter=10000, dual = False).fit(X_train_poly, y_train)

    for i, m in enumerate(models):
        y_pred = m.predict(X_test)
        accs[i] += acc(y_test, y_pred)

    y_pred_rbf = model_rbf.predict(X_test_rbf)
    accs[3] += acc(y_test, y_pred_rbf)

    y_pred_poly = model_ts.predict(X_test_poly)
    accs[4] += acc(y_test, y_pred_poly)
names = ["Linear SVM", "RBF SVM", "Polynomial SVM", "Explicit RBF", "Explicit Polynomial"]  
for n, a in zip(names, accs):
    print(n + ": " + str(np.round(a/nr, 3)))

/home/koki/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/koki/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/koki/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/koki/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/koki/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the numbe

Linear SVM: 0.985
RBF SVM: 0.817
Polynomial SVM: 0.998
Explicit RBF: 0.818
Explicit Polynomial: 0.997
